In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, accuracy_score
from collections import OrderedDict

pd.set_option('display.max_columns', None)
path = '/Users/aaronng/Downloads/Thesis/'

seed=123
dataset_suffix = ''
type_suffix = ''

np.random.seed(seed)
random.seed(seed)

In [2]:
# Load raw data prepared from data_prep.ipynb
raw_data = pd.read_csv(path + 'raw_data.csv')
raw_data['mode'] = np.where(raw_data['mode'] == 'major', 1, 0)
raw_data.shape

((1617643, 51), (400599, 51))

In [3]:
# Define 18 unique audio attributes
audio_cols = ['acousticness','beat_strength','bounciness','danceability','dyn_range_mean','energy','flatness','instrumentalness','key','liveness','loudness','mechanism','mode','organism','speechiness','tempo','time_signature','valence']
audio_cols.sort()

# Scale audio attributes
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(raw_data[audio_cols])
data = pd.DataFrame(data, columns=audio_cols)
data = pd.concat([raw_data.drop(audio_cols, axis=1), data], axis=1)
data

### Load audio states and top audio attributes

In [4]:
import itertools
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Load top audio attributes
true_preds = pd.read_csv(path + 'topN-features-{}{}.csv'.format(dataset_suffix,type_suffix)).sort_values('session_id').reset_index(drop=True)
true_preds = true_preds[(true_preds['top1'] != 'NONE')]

true_preds = true_preds.set_index('session_id')
sess_ids_filt = true_preds.index

true_preds

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18
session_id,,,,,,,,,,,,,,,,,,
36_00042572-ee58-48eb-a6fc-675bda0bbb98,beat_strength,acousticness,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_0004794d-dca2-426a-b4d1-0f5fdbd0c1a4,flatness,danceability,mode,energy,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_0007583d-ca87-4edc-8282-987bd32c95be,valence,acousticness,mechanism,energy,loudness,organism,tempo,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_00095e9c-b1c4-4040-874f-fedc5838ce62,valence,liveness,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_0009b5af-9798-4567-a7e7-03a8c701655f,acousticness,mode,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_ffd6d90d-f140-416c-80f1-30bef230e5ee,valence,acousticness,tempo,organism,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_ffda3299-c4d0-4683-9f6c-db8d9fdebc51,beat_strength,bounciness,flatness,speechiness,tempo,danceability,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
36_ffea5687-1726-4cc8-9131-20a4360be28d,beat_strength,speechiness,acousticness,mode,organism,key,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [6]:
true_preds_top1 = true_preds[['top1']]
true_preds_top2 = true_preds[['top1','top2']]
true_preds_top3 = true_preds[['top1','top2','top3']]

In [9]:
# Load raw data
data = data[data['session_id'].isin(sess_ids_filt)][['session_id','session_position','skip_2']+audio_cols].reset_index(drop=True)
data['skip_2'] = data['skip_2'].astype(int)
data['relevance'] = data['skip_2'] ^ 1
data = data.set_index('session_id').drop('skip_2',axis=1)
data

,session_position,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,relevance
session_id,,,,,,,,,,,,,,,,,,,,
36_00042572-ee58-48eb-a6fc-675bda0bbb98,1,0.946204,0.235728,0.356757,0.337399,0.232456,0.230894,0.937744,3.562460e-07,0.090909,0.187859,0.832065,0.284091,1.0,0.864734,0.051175,0.692732,1.0,0.428736,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,2,0.079668,0.618124,0.644286,0.797963,0.300100,0.758892,0.893822,1.332862e-09,0.545455,0.080520,0.830038,0.925806,1.0,0.079374,0.054439,0.451819,0.8,0.855310,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,3,0.012234,0.843084,0.890321,0.880344,0.443435,0.547475,0.925688,1.506017e-05,0.636364,0.028050,0.839986,0.805471,1.0,0.142431,0.185094,0.406699,0.8,0.687459,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,4,0.376191,0.837427,0.852521,0.834778,0.389007,0.484764,0.916195,1.466622e-05,0.909091,0.118421,0.824948,0.758123,0.0,0.325850,0.154805,0.328623,0.8,0.622393,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,5,0.023394,0.420738,0.462627,0.605105,0.247103,0.658409,0.925441,6.004185e-05,0.090909,0.237237,0.836443,0.580488,1.0,0.307031,0.057466,0.595593,0.8,0.267841,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,16,0.376191,0.837427,0.852521,0.834778,0.389007,0.484764,0.916195,1.466622e-05,0.909091,0.118421,0.824948,0.758123,0.0,0.325850,0.154805,0.328623,0.8,0.622393,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,17,0.047791,0.570793,0.590077,0.754896,0.279471,0.764175,0.908482,9.440119e-05,0.090909,0.054322,0.860191,0.815700,1.0,0.139095,0.039102,0.500840,0.8,0.581523,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,18,0.558264,0.796981,0.862822,0.924597,0.433919,0.536717,0.914334,4.037161e-03,0.818182,0.101738,0.839707,0.911017,0.0,0.411408,0.082855,0.525689,0.8,0.710777,1


In [10]:
# Load extracted audio states
states_full = pd.read_csv(path + 'states-{}{}.csv'.format(dataset_suffix,type_suffix))
states = states_full[states_full['session_id'].isin(sess_ids_filt)]
states = states.set_index('session_id')
states

,session_position,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,relevance
session_id,,,,,,,,,,,,,,,,,,,,
36_00042572-ee58-48eb-a6fc-675bda0bbb98,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,4,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,16,0,2,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,17,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,18,0,2,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1


### Rank by User Relevance Metrics (i.e. ranking score in current state)

In [13]:
K=10
N=5

In [14]:
from sklearn.metrics import f1_score

# Metric definitions
# https://gist.github.com/bwhite/3726239/
def dcg_score(y_score, k=999):
    y = y_score[:k]
    score = 0
    
    for i in range(len(y)):
        score += (2**y[i] - 1) / np.log2(i+2)

    return np.sum(score)

def ndcg_score(y_score, k=999):
    y = y_score[:k]
    
    idcg = dcg_score(sorted(y_score, reverse=True), k)
    return dcg_score(y_score, k) / idcg if idcg > 0 else 0

def precision_score(y_score, k=999):
    cumulative_score = 0
    num_pos = 0
    y = y_score[:k]
    
    return np.sum(y) / len(y)

def average_precision_score(y_score, k=999):
    cumulative_score = 0
    num_pos = 0
    y = y_score[:k]
    
    for i in range(len(y)):
        if y[i] == 1:
            num_pos += 1
            cumulative_score += num_pos / (i+1)
    
    return cumulative_score / num_pos if num_pos > 0 else 0

def reciprocal_rank(y_score, k=999):
    y = y_score[:k]
    
    try:
        return 1/(y.index(1)+1)
    except ValueError:
        return 0

In [15]:
def calc_current_metrics(data, K=10, N=5):
    metrics = []
    count=0

    for session_id in data.index.unique():
        session_data = data.loc[session_id]
        session_data = session_data[session_data['session_position'] > N]
        y = list(session_data.reset_index(drop=True)['relevance'])
        
        ndcg_K = ndcg_score(y, K)
        ap_K = average_precision_score(y, K)
        p_K = precision_score(y, K)
        rr_K = reciprocal_rank(y, K)
        
        metrics.append([session_id, ndcg_K, ap_K, p_K, rr_K])

        count+=1
        if count%100 == 0:
            print('Processed {} sessions'.format(count))
            
    return pd.DataFrame(metrics, columns=[
        'session_id',
        'NDCG@{}'.format(K), 
        'AP@{}'.format(K),
        'P@{}'.format(K), 
        'RR@{}'.format(K)
    ])

In [ ]:
current_metrics = calc_current_metrics(test[['session_position','relevance']], N=N)
current_metrics

In [17]:
current_metrics.mean()

NDCG@10    0.659390
AP@10      0.604511
P@10       0.513857
RR@10      0.618677
dtype: float64

### Known Top3 Features metrics

In [182]:
scaler = MinMaxScaler()

# Method to re-rank tracks using audio states and calculate metrics
def calc_pred_metrics(states, feats, data, K=10, N=5, max_states=3):
    metrics = []
    count=0

    for session_id in states.index.unique():
        session_states = states.loc[session_id].reset_index(drop=True)
        session_topN_feats = list(feats.loc[session_id].values)
        session_topN_feats = [feat for feat in session_topN_feats if feat != 'NONE']
        session_topN_feats = list(OrderedDict.fromkeys(session_topN_feats)) # Remove duplicate topN features

        session_firstN = session_states[session_states['session_position'] <= N][session_topN_feats + ['relevance']]
        rel_tracks = np.sum(list(session_firstN['relevance']))
        
        # Replace non-relevant tracks (0) with -1 score, relevant tracks (1) remain as score of 1
        session_firstN['score'] = session_firstN['relevance'].replace(0,-1)
        session_remaining = session_states[session_states['session_position'] > N][session_topN_feats + ['session_position','relevance']]
        
        for feat in session_topN_feats:
            feat_scores_dict = session_firstN[[feat, 'score']].groupby(feat).sum().sort_values('score', ascending=False).to_dict()['score']
            
            # Replace best state with score of 1, rest 0
            best_state = list(feat_scores_dict.keys())[0]
            feat_scores_dict = {best_state: 1}
            
            # Fill rest of dict (unseen states) with score of 0
            for i in range(max_states):
                if i not in feat_scores_dict:
                    feat_scores_dict[i]=0
                    
            session_remaining[feat+'_score'] = session_remaining[feat].replace(feat_scores_dict)
        
        feat_score_cols = [col for col in session_remaining.columns if '_score' in col]
        session_remaining.insert(len(session_remaining.columns), 'score', session_remaining[feat_score_cols].sum(axis=1))
        session_remaining = session_remaining.sort_values(['score','session_position'], ascending=[False,True])

        pred = list(session_remaining['relevance'])
        ndcg_K_i = ndcg_score(pred, K)
        ap_K_i = average_precision_score(pred, K)
        p_K_i = precision_score(pred, K)
        rr_K_i = reciprocal_rank(pred, K)
         
        metrics.append([session_id, ndcg_K_i, ap_K_i, p_K_i, rr_K_i, np.sum(pred), len(pred)])
        
        count+=1
        if count%10 == 0:
            print('Processed {} sessions'.format(count))
    
    topN = feats.values.shape[1]
    return pd.DataFrame(metrics, columns=[
        'session_id',
        'NDCG@{}_top{}'.format(K, topN), 
        'AP@{}_top{}'.format(K, topN),
        'P@{}_top{}'.format(K, topN), 
        'RR@{}_top{}'.format(K, topN),
        'Relevant remaining','Length remaining'
    ])

In [ ]:
true_pred_metrics1 = calc_pred_metrics(states, true_preds_top1, test, N=N)
true_pred_metrics1

In [27]:
true_pred_metrics1.mean()

NDCG@10_top1    0.725907
AP@10_top1      0.674701
P@10_top1       0.528984
RR@10_top1      0.700534
dtype: float64

In [ ]:
true_pred_metrics2 = calc_pred_metrics(states, true_preds_top2, test, N=N)
true_pred_metrics2

In [29]:
true_pred_metrics2.mean()

NDCG@10_top2    0.731816
AP@10_top2      0.676165
P@10_top2       0.531923
RR@10_top2      0.715563
dtype: float64

In [ ]:
true_pred_metrics3 = calc_pred_metrics(states, true_preds_top3, test, N=N)
true_pred_metrics3

In [31]:
true_pred_metrics3.mean()

NDCG@10_top3    0.731329
AP@10_top3      0.675899
P@10_top3       0.531376
RR@10_top3      0.717795
dtype: float64

In [32]:
true_pred_metrics = copy.deepcopy(true_pred_metrics1)
true_pred_metrics = true_pred_metrics.merge(true_pred_metrics2, on='session_id')
true_pred_metrics = true_pred_metrics.merge(true_pred_metrics3, on='session_id')
true_pred_metrics

,session_id,NDCG@10_top1,AP@10_top1,P@10_top1,RR@10_top1,NDCG@10_top2,AP@10_top2,P@10_top2,RR@10_top2,NDCG@10_top3,AP@10_top3,P@10_top3,RR@10_top3
0,36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.919721,0.833333,0.400000,1.000000,0.877215,0.750000,0.400000,1.00,0.877215,0.750000,0.400000,1.000000
1,36_0004794d-dca2-426a-b4d1-0f5fdbd0c1a4,0.184576,0.111111,0.100000,0.111111,0.177239,0.100000,0.100000,0.10,0.177239,0.100000,0.100000,0.100000
2,36_0007583d-ca87-4edc-8282-987bd32c95be,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000
3,36_00095e9c-b1c4-4040-874f-fedc5838ce62,0.306574,0.333333,0.100000,0.333333,0.482476,0.291667,0.200000,0.25,0.482476,0.291667,0.200000,0.250000
4,36_0009b5af-9798-4567-a7e7-03a8c701655f,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22280,36_ffd6d90d-f140-416c-80f1-30bef230e5ee,0.530721,0.583333,0.200000,0.500000,0.530721,0.583333,0.200000,0.50,0.765361,1.000000,0.200000,1.000000
22281,36_ffda3299-c4d0-4683-9f6c-db8d9fdebc51,0.904717,0.804167,0.571429,1.000000,0.904717,0.804167,0.571429,1.00,0.904717,0.804167,0.571429,1.000000
22282,36_ffea5687-1726-4cc8-9131-20a4360be28d,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000
22283,36_ffed5604-4660-46bb-9913-46717d06c3e4,0.498189,0.500000,0.200000,0.500000,0.498189,0.500000,0.200000,0.50,0.578014,0.388889,0.300000,0.333333


### Global Top3 Features Metrics

In [142]:
global_features_top3 = copy.deepcopy(preds_top3)
global_features_top3['top1'] = 'key'
global_features_top3['top2'] = 'acousticness'
global_features_top3['top3'] = 'beat_strength'
global_features_top3

,top1,top2,top3
session_id,,,
36_00228707-4619-45af-ab63-de45d5896db1,key,acousticness,beat_strength
36_00265425-9562-4a6c-87cb-febc8fc61997,key,acousticness,beat_strength
36_00582287-61ad-4fce-8d1d-88bc697efef9,key,acousticness,beat_strength
36_005b98e0-e22c-45e1-b317-d2308d5dc619,key,acousticness,beat_strength
36_0068b5e0-2a31-46e7-bd41-5666e83ad482,key,acousticness,beat_strength
...,...,...,...
36_ffc522eb-ecb8-4d6e-acbd-4afc4b28c099,key,acousticness,beat_strength
36_ffca4f91-7841-448d-be16-8027f6049544,key,acousticness,beat_strength
36_ffcf3a10-dec3-4078-bc07-095961ec2224,key,acousticness,beat_strength


In [ ]:
global_metrics1 = calc_pred_metrics(states, global_features_top3[['top1']], test, N=N, with_cos_sim=False)
global_metrics1

In [144]:
global_metrics1.mean()

NDCG@10_top1    0.619520
AP@10_top1      0.566086
P@10_top1       0.475062
RR@10_top1      0.582151
dtype: float64

In [ ]:
global_metrics2 = calc_pred_metrics(states, global_features_top3[['top1','top2']], test, N=N, with_cos_sim=False)
global_metrics2

In [146]:
global_metrics2.mean()

NDCG@10_top2    0.621753
AP@10_top2      0.567297
P@10_top2       0.475237
RR@10_top2      0.588687
dtype: float64

In [ ]:
global_metrics3 = calc_pred_metrics(states, global_features_top3, test, N=N, with_cos_sim=False)
global_metrics3

In [148]:
global_metrics3.mean()

NDCG@10_top3    0.623434
AP@10_top3      0.568487
P@10_top3       0.475462
RR@10_top3      0.593668
dtype: float64

In [149]:
global_metrics = copy.deepcopy(global_metrics1)
global_metrics = global_metrics.merge(global_metrics2, on='session_id')
global_metrics = global_metrics.merge(global_metrics3, on='session_id')
global_metrics

,session_id,NDCG@10_top1,AP@10_top1,P@10_top1,RR@10_top1,NDCG@10_top2,AP@10_top2,P@10_top2,RR@10_top2,NDCG@10_top3,AP@10_top3,P@10_top3,RR@10_top3
0,36_00228707-4619-45af-ab63-de45d5896db1,0.501266,0.325000,0.400000,0.250000,0.501266,0.325000,0.400000,0.250000,0.501266,0.325000,0.400000,0.250000
1,36_00265425-9562-4a6c-87cb-febc8fc61997,0.301030,0.111111,0.100000,0.111111,0.356207,0.166667,0.100000,0.166667,0.356207,0.166667,0.100000,0.166667
2,36_00582287-61ad-4fce-8d1d-88bc697efef9,0.826333,0.875546,0.800000,1.000000,0.826333,0.875546,0.800000,1.000000,0.826333,0.875546,0.800000,1.000000
3,36_005b98e0-e22c-45e1-b317-d2308d5dc619,0.455605,0.266667,0.200000,0.200000,0.455605,0.266667,0.200000,0.200000,0.455605,0.266667,0.200000,0.200000
4,36_0068b5e0-2a31-46e7-bd41-5666e83ad482,0.642596,0.502183,0.666667,0.250000,0.642596,0.502183,0.666667,0.250000,0.642596,0.502183,0.666667,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,36_ffc522eb-ecb8-4d6e-acbd-4afc4b28c099,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3996,36_ffca4f91-7841-448d-be16-8027f6049544,0.583372,0.542347,0.700000,0.333333,0.583372,0.542347,0.700000,0.333333,0.583372,0.542347,0.700000,0.333333
3997,36_ffcf3a10-dec3-4078-bc07-095961ec2224,0.955830,0.887500,0.800000,1.000000,0.955830,0.887500,0.800000,1.000000,0.904717,0.804167,0.800000,1.000000
3998,36_ffe5f35f-21c4-47c7-9675-3c0f4c6c8f35,0.455605,0.266667,0.333333,0.200000,0.455605,0.266667,0.333333,0.200000,0.455605,0.266667,0.333333,0.200000


### Random Top3 Features Metrics

In [ ]:
random_features_top3 = copy.deepcopy(preds_top3)
random_features_top3['top1'] = random.choice(audio_cols)
random_features_top3['top2'] = random.choice(audio_cols)
random_features_top3['top3'] = random.choice(audio_cols)
random_features_top3

In [ ]:
random_metrics1 = calc_pred_metrics(states, random_features_top3[['top1']], test, N=N, with_cos_sim=False)
random_metrics1

In [94]:
random_metrics1.mean()

NDCG@10_top1    0.580837
AP@10_top1      0.529924
P@10_top1       0.458847
RR@10_top1      0.545199
dtype: float64

In [ ]:
random_metrics2 = calc_pred_metrics(states, random_features_top3[['top1','top2']], test, N=N, with_cos_sim=False)
random_metrics2

In [ ]:
random_metrics2.mean()

In [ ]:
random_metrics3 = calc_pred_metrics(states, random_features_top3, test, N=N, with_cos_sim=False)
random_metrics3

In [98]:
random_metrics3.mean()

NDCG@10_top3    0.586104
AP@10_top3      0.534386
P@10_top3       0.460611
RR@10_top3      0.551590
dtype: float64

In [99]:
random_top3_metrics = copy.deepcopy(random_metrics1)
random_top3_metrics = random_top3_metrics.merge(random_metrics2, on='session_id')
random_top3_metrics = random_top3_metrics.merge(random_metrics3, on='session_id')
random_top3_metrics

,session_id,NDCG@10_top1,AP@10_top1,P@10_top1,RR@10_top1,NDCG@10_top2,AP@10_top2,P@10_top2,RR@10_top2,NDCG@10_top3,AP@10_top3,P@10_top3,RR@10_top3
0,34_000c3a29-e81b-4b32-9047-94d0995a479a,0.951231,0.876667,0.625,1.00,0.951231,0.876667,0.625,1.00,0.918216,0.796190,0.625,1.00
1,34_001b51f4-1731-43c6-b055-baccf52ac622,0.430677,0.250000,0.100,0.25,0.430677,0.250000,0.100,0.25,0.630930,0.500000,0.100,0.50
2,34_0035f167-f08a-47ff-bb6f-3fdba1626c3b,0.921787,0.809524,0.300,1.00,0.921787,0.809524,0.300,1.00,0.921787,0.809524,0.300,1.00
3,34_00603512-23b6-4363-90bb-66f4c37d8e91,1.000000,1.000000,1.000,1.00,1.000000,1.000000,1.000,1.00,1.000000,1.000000,1.000,1.00
4,34_00622d99-c6e7-428a-9e6e-d5ffa7ae37d8,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5720,34_ffd6f828-b023-44ba-9a06-19367642f68c,0.977015,0.928263,0.900,1.00,0.977015,0.928263,0.900,1.00,0.977015,0.928263,0.900,1.00
5721,34_ffdabfc7-36cd-4c0c-9ede-b877c8a5be64,0.543354,0.365476,0.500,0.20,0.572425,0.394643,0.500,0.25,0.572425,0.394643,0.500,0.25
5722,34_ffe3e2f7-b73b-4c62-81a1-c810ffdc6655,0.927961,0.812500,0.500,1.00,0.927961,0.812500,0.500,1.00,0.927961,0.812500,0.500,1.00
5723,34_ffe6a788-cc50-456d-b5db-3e6a809a9001,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.00


### Rank by similarity Metrics

In [33]:
test_firstN = data[data['session_position'].isin(range(1,N+1))]
test_remain = data[data['session_position'].isin(range(N+1,N+99))]
print(test_firstN.shape, test_remain.shape)

(111425, 20) (250301, 20)


In [34]:
test_firstN_agg = test_firstN.drop(['session_position', 'relevance'], axis=1).reset_index().groupby(['session_id']).mean().reset_index()
test_merged = test_firstN_agg.merge(test_remain.reset_index(), left_on='session_id', right_on='session_id')
test_merged = test_merged.set_index('session_id')
test_merged

,acousticness_x,beat_strength_x,bounciness_x,danceability_x,dyn_range_mean_x,energy_x,flatness_x,instrumentalness_x,key_x,liveness_x,loudness_x,mechanism_x,mode_x,organism_x,speechiness_x,tempo_x,time_signature_x,valence_x,session_position,acousticness_y,beat_strength_y,bounciness_y,danceability_y,dyn_range_mean_y,energy_y,flatness_y,instrumentalness_y,key_y,liveness_y,loudness_y,mechanism_y,mode_y,organism_y,speechiness_y,tempo_y,time_signature_y,valence_y,relevance
session_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.287538,0.59102,0.641303,0.691118,0.322420,0.536087,0.919778,0.000018,0.454545,0.130417,0.832696,0.670796,0.8,0.343884,0.100596,0.495093,0.84,0.572348,6,0.249331,0.541151,0.561132,0.724333,0.270499,0.459354,0.925172,5.281906e-10,0.363636,0.212495,0.829110,0.565097,1.0,0.365949,0.032933,0.492446,0.8,0.459944,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.287538,0.59102,0.641303,0.691118,0.322420,0.536087,0.919778,0.000018,0.454545,0.130417,0.832696,0.670796,0.8,0.343884,0.100596,0.495093,0.84,0.572348,7,0.107147,0.673868,0.728439,0.850963,0.344360,0.603374,0.905327,7.401743e-06,0.000000,0.105539,0.856973,0.685950,0.0,0.242375,0.040933,0.468344,0.8,0.735236,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.287538,0.59102,0.641303,0.691118,0.322420,0.536087,0.919778,0.000018,0.454545,0.130417,0.832696,0.670796,0.8,0.343884,0.100596,0.495093,0.84,0.572348,8,0.187034,0.581245,0.613490,0.712931,0.291493,0.614444,0.917237,4.933471e-07,0.909091,0.097369,0.823711,0.753425,0.0,0.225810,0.044425,0.608410,0.8,0.657861,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.287538,0.59102,0.641303,0.691118,0.322420,0.536087,0.919778,0.000018,0.454545,0.130417,0.832696,0.670796,0.8,0.343884,0.100596,0.495093,0.84,0.572348,9,0.021163,0.420022,0.401185,0.570154,0.212269,0.656503,0.915210,1.368348e-05,0.727273,0.213270,0.844503,0.447552,1.0,0.403995,0.044582,0.410867,0.8,0.364230,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,0.287538,0.59102,0.641303,0.691118,0.322420,0.536087,0.919778,0.000018,0.454545,0.130417,0.832696,0.670796,0.8,0.343884,0.100596,0.495093,0.84,0.572348,10,0.040149,0.507960,0.521238,0.702461,0.256256,0.713157,0.910093,3.108230e-06,0.818182,0.294377,0.843049,0.625000,0.0,0.275584,0.061160,0.500984,0.8,0.353742,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,0.089194,0.59909,0.641818,0.751581,0.321405,0.628982,0.917368,0.000016,0.600000,0.213067,0.843055,0.608283,0.8,0.298193,0.157934,0.494262,0.80,0.501015,16,0.376191,0.837427,0.852521,0.834778,0.389007,0.484764,0.916195,1.466622e-05,0.909091,0.118421,0.824948,0.758123,0.0,0.325850,0.154805,0.328623,0.8,0.622393,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,0.089194,0.59909,0.641818,0.751581,0.321405,0.628982,0.917368,0.000016,0.600000,0.213067,0.843055,0.608283,0.8,0.298193,0.157934,0.494262,0.80,0.501015,17,0.047791,0.570793,0.590077,0.754896,0.279471,0.764175,0.908482,9.440119e-05,0.090909,0.054322,0.860191,0.815700,1.0,0.139095,0.039102,0.500840,0.8,0.581523,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,0.089194,0.59909,0.641818,0.751581,0.321405,0.628982,0.917368,0.000016,0.600000,0.213067,0.843055,0.608283,0.8,0.298193,0.157934,0.494262,0.80,0.501015,18,0.558264,0.796981,0.862822,0.924597,0.433919,0.536717,0.914334,4.037161e-03,0.818182,0.101738,0.839707,0.911017,0.0,0.411408,0.082855,0.525689,0.8,0.710777,1


In [ ]:
x_cols = [col for col in test_merged.columns if '_x' in col]
y_cols = [col for col in test_merged.columns if '_y' in col]
cos_sim_scores = []
count=0

for i, row in test_merged.iterrows():
    cos_sim_scores.append(cosine_similarity(row[x_cols].values.reshape(1, -1), row[y_cols].values.reshape(1, -1))[0][0])
    
    count+=1
    if count%100 == 0:
        print('Processed {} rows'.format(count))

In [37]:
test_merged['score'] = cos_sim_scores

In [38]:
def calc_metrics_by_col(data, col, K=10):
    metrics = []
    count=0

    for session_id in data.index.unique():
        session_data = data.loc[session_id]
        session_data = session_data.sort_values([col, 'session_position'], ascending=[False,True])
        pred = list(session_data.reset_index(drop=True)['relevance'])
        
        # print(session_data)
        
        ap_K = average_precision_score(pred, K)
        ndcg_K = ndcg_score(pred, K)
        p_K = precision_score(pred, K)
        rr_K = reciprocal_rank(pred, K)

        metrics.append([session_id, ndcg_K, ap_K, p_K, rr_K])

        count+=1
        if count%100 == 0:
            print('Processed {} sessions'.format(count))
            
    return pd.DataFrame(metrics, columns=[
        'session_id',
        'NDCG@{}'.format(K), 
        'AP@{}'.format(K),
        'P@{}'.format(K), 
        'RR@{}'.format(K)
    ])

In [ ]:
similarity_metrics = calc_metrics_by_col(test_merged, 'score')
similarity_metrics

In [40]:
similarity_metrics.mean()

NDCG@10    0.691085
AP@10      0.629450
P@10       0.522320
RR@10      0.676748
dtype: float64

### Rank by popularity Metrics

In [41]:
test_with_pop = data[data['session_id'].isin(sess_ids_filt)][['session_id','session_position','us_popularity_estimate']].reset_index(drop=True)
test_with_pop['relevance'] = states['relevance'].values
test_with_pop = test_with_pop.set_index('session_id')
test_with_pop = test_with_pop[test_with_pop['session_position'] > N]
test_with_pop

,session_position,us_popularity_estimate,relevance
session_id,,,
36_00042572-ee58-48eb-a6fc-675bda0bbb98,6,99.999717,1
36_00042572-ee58-48eb-a6fc-675bda0bbb98,7,99.999107,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,8,99.999635,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,9,99.999942,0
36_00042572-ee58-48eb-a6fc-675bda0bbb98,10,99.999897,1
...,...,...,...
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,16,99.999569,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,17,97.270832,1
36_fffe80a7-1db6-4842-b2a6-6f7165f40f2b,18,99.999989,1


In [ ]:
popularity_metrics = calc_metrics_by_col(test_with_pop, 'us_popularity_estimate')
popularity_metrics

In [44]:
popularity_metrics.mean()

NDCG@10    0.711652
AP@10      0.651837
P@10       0.526574
RR@10      0.702608
dtype: float64

### Metrics compared

In [ ]:
metrics_comp = []
metric_cols = ['NDCG@10', 'AP@10', 'P@10', 'RR@10']

for col in metric_cols:
    tmp = []
    tmp.append([current_metrics[col].mean(), -1, -1])
    tmp.append([similarity_metrics[col].mean(), -1, -1])
    tmp.append([popularity_metrics[col].mean(), -1, -1])
    tmp.append([random_top3_metrics[col+'_top1'].mean(), random_top3_metrics[col+'_top2'].mean(), random_top3_metrics[col+'_top3'].mean()])
    tmp.append([global_metrics[col+'_top1'].mean(), global_metrics[col+'_top2'].mean(), global_metrics[col+'_top3'].mean()])
    tmp.append([true_pred_metrics[col+'_top1'].mean(), true_pred_metrics[col+'_top2'].mean(), true_pred_metrics[col+'_top3'].mean()])
    metrics_comp = np.array(tmp) if len(metrics_comp) == 0 else np.concatenate((metrics_comp, tmp), axis=1)

columns = [[col for col in metric_cols for i in range(3)], ['Top 1', 'Top 2', 'Top 3']*4]
metrics_comp = pd.DataFrame(metrics_comp, columns=columns)
metrics_comp = metrics_comp.applymap(lambda x: "{0:,.3f}".format(x) if x > -1 else '')
metrics_comp.index = ['Current', 'Similarity', 'Popularity', 'Random Top3', 'Global Top3', 'Known Top3']
metrics_comp